In [27]:
import tatqa_utils
import pandas as pd

In [28]:
devdf = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [29]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [31]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [37]:
def gen_code(llm, question, table): 
    prompt =  f"Generate a Python code that can answer the following question using the following table! The code must contain only one function called 'run', and no wrapping class. The function must return (value, scale) where value is numeric float with accuracy to two decimal places and scale is thousand, million, billion, percent or empty string. Do not write explanation, just code.\nQuestion: {question} \n Table: {table}"
    prompt = remove_thousand_separators(prompt)
    prompt = replace_bracketed_numbers(prompt)
    res = llm.invoke(prompt)
    code = res.content.replace('```python','').replace('```','')
    return (prompt, code)

def exec_code(code, table):  
    table = remove_thousand_separators(f"{table}")
    table = replace_bracketed_numbers(table)
    code = code.replace("* 1000000","").replace("*1000000","").replace("/ 1000000","").replace("/1000000","").replace("/ 1000","").replace("/1000","")
    code = code.replace('abs(','(')
    try: 
        loc = locals()   
        if not "run()" in code:
            exec(code + f"\nr = run({table})\n", globals(), loc)
        else: 
            exec(code + "\nr = run()\n", globals(), loc)
        return loc['r']
    except Exception as e:
            s = '[Error]'+ str(e)
            print(s)
            return (s,'')
#table = "[['', '', 'Years Ended September 30,', ''], ['', '2019', '2018', '2017'], ['Fixed Price', '$  1,452.4', '$  1,146.2', '$  1,036.9'], ['Other', '44.1', '56.7', '70.8'], ['Total sales', '$1,496.5', '$1,202.9', '$1,107.7']]"
#question = 'What is the change in Fixed Price in 2019 from 2018?'
table = "[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]"
#table = remove_thousand_separators(table)
print(table)
question = 'What was the percentage change in Value added tax receivables, net, noncurrent in 2019 from 2018?'
( prompt, code) = gen_code(llm, question, table)
res = exec_code(code, table)
print(code, res)

[['', 'December 31,', ''], ['', '2019', '2018'], ['Trade accounts receivable, net, noncurrent (Note 2)', '$26,496', '$15,948'], ['Equity method investments (Note 1)', '9,254', '9,702'], ['Net deferred tax assets, noncurrent (Note 20)', '6,774', '5,797'], ['Rent and other deposits', '6,106', '5,687'], ['Value added tax receivables, net, noncurrent', '592', '519'], ['Other', '6,723', '5,711'], ['', '$55,945', '$43,364']]
[Error]invalid syntax. Perhaps you forgot a comma? (<string>, line 11)
def run():
    current_year = 2019
    previous_year = 2018
    
    current_value = 592
    previous_value = 519
    
    percentage_change = ((current_value - previous_value) / previous_value) * 100
    
    if percentage_change > 1000000000:
        return (round(percentage_change / 1000000000, 2), 'billion')
    elif percentage_change > 1000000:
        return (round(percentage_change / 1000000, 2), 'million')
    elif percentage_change > 1000:
        return (round(percentage_change / 1000, 2), '

In [33]:
from tqdm.notebook import tqdm as log_progress

res = []
for i, item in log_progress(devdf.iterrows()):
    table = item['table']['table']
    #table = transform_elements(table, remove_thousand_separators)
    #table = [remove_thousand_separators(cell) for row in table for cell in row]
    #table = [cell.replace('$', '') for row in table for cell in row]
    #print (table)
    for q in item['questions']:        
        if q['answer_type'] == 'arithmetic' and 'table' in q['answer_from']:             
            ( prompt, code)  = gen_code(llm, q['question'], table)
            r = exec_code(code, table)
            err =  None
            if r is None:
                (pred_value, pred_unit) = (None, None)
            else:   
                if len(r) == 2:
                    (pred_value, pred_unit) = r          
                    if pred_unit == "%" or :
                        pred_unit = 'percent'
                    if pred_unit not in ["", 'thousand', 'million', 'billion', 'percent']:
                        print('Invalid ', pred_unit)
                        pred_unit = ""    
               
                    #print(pred_value, type(pred_value))
                    if isinstance(pred_value, str) and  pred_value.startswith('[Error]'):
                        (pred_value, pred_unit) = (None, None)
                        err,_ = r                                                        
                        
                elif len(r) == 1:
                    pred_value = r                
                else:
                    (pred_value, pred_unit) = (None, None)        
                
            #res.append({"table":table, "q":q, "pred":pred, "code": code})
            res.append(({"answer_type":q["answer_type"], "answer": q["answer"], 'scale': q["scale"]}, pred_value, pred_unit, q, code, prompt, item['table'], err))


0it [00:00, ?it/s]

Invalid  value
Invalid  millions
Invalid  percentage
Invalid  percentage
(6.67, 'percentage')
Invalid  percentage
Invalid  percentage
Invalid  €m
Invalid  €m
Invalid  €m
Invalid  percentage
Invalid  percentage
Invalid  percentage
(2.93, 'ratio')
Invalid  ratio
Invalid  value
Invalid  Value added tax receivables, net, noncurrent
Invalid  percentage
Invalid  €m
Invalid  €m
Invalid  €m
Invalid  USD
Invalid  float
Invalid  millions
Invalid  percentage
Invalid  percentage
(399.33, 'million')
Invalid  £m
(0.51, '%')
Invalid  ratio
Invalid  millions
Invalid  millions
[Error]could not convert string to float: '$17900'
Invalid  USD
Invalid  USD
Invalid  percentage
(-9.9, 'Preferred stock disposition')
Invalid  Preferred stock disposition
(66.22, '%')
(70056, 'USD')
Invalid  USD
Invalid  value
Invalid  percentage
Invalid  Millions
Invalid  Millions
Invalid  €m
Invalid  €m
Invalid  thousands
(95.5, '%')
(4.0, 'per employee')
Invalid  per employee
Invalid  percentage
Invalid  scale
Invalid  USDm
I

KeyboardInterrupt: 

In [34]:
from tatqa_metric import TaTQAEmAndF1

metrics = TaTQAEmAndF1()

for ans, pred, pred_scale, _,_, _,_,_ in res:
    metrics(ans, pred, pred_scale)
pred_em, pred_f1, scale_score, op_score = metrics.get_overall_metric(reset=True)
print( pred_em, pred_f1, scale_score)

0.19811320754716982 0.19811320754716982 0.23113207547169812


In [19]:
import re 
def remove_thousand_separators(text):
    # This regular expression matches numbers with commas as thousand separators
    return re.sub(r'(?<=\d),(?=\d{3})', '', text)
    
def transform_elements(list_of_lists, transform_func):    
    return [
        [transform_func(item) for item in sublist]
        for sublist in list_of_lists
    ]
    
def replace_bracketed_numbers(text):
    return re.sub(r"\((\d+)\)", lambda x: f"-{x.group(1)}", text)

In [36]:
good = 0
for ans, pred, pred_scale, q,code, _,table,err in res:
    llimit = ans['answer']*0.99
    ulimit = ans['answer']*1.01
    
    if pred  is None or isinstance(pred, tuple) or isinstance(pred, str):
        #type error
        #if isinstance(pred, str):
        #    print (pred, pred_scale, ' -- ' , ans['answer'])
        #if isinstance(pred, tuple):
        #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        #print('===============================================================================')
        print(err)
        continue
    
    pred = round(pred, 2)
    
    if (pred > 0 and llimit < pred and pred < ulimit) or (pred < 0 and llimit > pred and pred > ulimit) or pred==ulimit or pred == llimit:
        good = good + 1
        if  ans['scale'] != pred_scale:
            print (q['uid'], pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
          
    else:
        #value error
        #print (q['uid'], pred, pred_scale, ' <> ' , ans['answer'], ans['scale'])
        None
        #print(q['question'])
        #print(table['table'])
        #print(code)
        
    #if isinstance(pred, str):
    #    print (pred, pred_scale, ' -- ' , ans['answer'])
    #if isinstance(pred, tuple):
    #    print (pred, pred_scale, ' == ' , ans['answer'], ans['scale'])
        
print (good/len(res))

eb787966-fa02-401f-bfaf-ccabf3828b23 -12.6   ==  -12.6 million
None
b2786c1a-37de-4120-b03c-32bf5c81f157 -94   ==  -94 million
fe11f001-3bfe-4089-8108-412676f0a780 -12.14   ==  -12.14 percent
5103aed0-b4e8-4fae-bf78-e2c9f4ba84cf 2.1   ==  2.1 percent
4dc8be43-d8d9-4b08-9ffd-9c19012361ce 6.67   ==  6.67 percent
6c44a1a8-0785-43a0-90ab-7e21df2c57d9 -8.11   ==  -8.11 percent
bed1fce2-69cb-4d1e-a34a-01950a1770bd 26.82   ==  26.82 percent
348d031d-73ab-4b35-af46-998cfef25775 25.1   ==  25.1 percent
aea850af-68cb-4cc1-80b2-c9c2860b273e 2.98   ==  2.98 percent
35d602ae-9131-4291-a30c-49a40f32bbe4 73.0   ==  73 thousand
a4f782f1-de3b-47a3-87af-1cb33c112b39 14.07   ==  14.07 percent
64c902c6-f426-4432-84b3-c10b3065716f -16458   ==  -16458 thousand
5d9b397d-16bb-4463-8f9a-b85507704a8d 154   ==  154 million
None
91812b92-5e94-414f-a447-4622aa3c2d10 1.57   ==  1.57 percent
65ec782c-691e-45df-b541-caecb85154ff 0.2   ==  0.2 million
bbe335ff-414f-48b3-8126-5a2c7c505de3 13237.4   ==  13237.4 million


In [25]:
res


(14.07, 'percentage')

In [81]:
good, len(res)

(135, 182)

In [159]:
def run():
    table = [['$ million', '2019', '2018', 'Change (%)'], ['Order intake1', '532.0', '470.0', '13.2'], ['Revenue', '503.6', '476.9', '5.6'], ['Gross profit', '368.6', '344.5', '7.0'], ['Gross margin (%)', '73.2', '72.2', '1.0'], ['Adjusted operating costs2', '275.7', '267.4', '3.1'], ['Adjusted operating profit2', '92.9', '77.1', '20.5'], ['Adjusted operating margin3 (%)', '18.4', '16.2', '2.2'], ['Reported operating profit', '88.6', '57.5', '54.1'], ['Effective tax rate4 (%)', '13.0', '15.4', '(2.4)'], ['Reported profit before tax', '89.6', '61.2', '46.4'], ['Adjusted basic earnings per share5 (cents)', '13.40', '10.86', '23.4'], ['Basic earnings per share (cents)', '12.79', '9.14', '39.9'], ['Free cash flow6', '100.1', '50.9', '96.7'], ['Closing cash', '183.2', '121.6', '50.7'], ['Final dividend per share7 (cents)', '3.45', '2.73', '26.4']]
    print(table)
    closing_cash_2019 = float(table[-1][1])
    closing_cash_2018 = float(table[-1][2])

    change_in_closing_cash = round(closing_cash_2019 - closing_cash_2018, 2)
    return (change_in_closing_cash, 'million')
run()

[['$ million', '2019', '2018', 'Change (%)'], ['Order intake1', '532.0', '470.0', '13.2'], ['Revenue', '503.6', '476.9', '5.6'], ['Gross profit', '368.6', '344.5', '7.0'], ['Gross margin (%)', '73.2', '72.2', '1.0'], ['Adjusted operating costs2', '275.7', '267.4', '3.1'], ['Adjusted operating profit2', '92.9', '77.1', '20.5'], ['Adjusted operating margin3 (%)', '18.4', '16.2', '2.2'], ['Reported operating profit', '88.6', '57.5', '54.1'], ['Effective tax rate4 (%)', '13.0', '15.4', '(2.4)'], ['Reported profit before tax', '89.6', '61.2', '46.4'], ['Adjusted basic earnings per share5 (cents)', '13.40', '10.86', '23.4'], ['Basic earnings per share (cents)', '12.79', '9.14', '39.9'], ['Free cash flow6', '100.1', '50.9', '96.7'], ['Closing cash', '183.2', '121.6', '50.7'], ['Final dividend per share7 (cents)', '3.45', '2.73', '26.4']]
3.45 3.45


(0.72, 'million')

In [157]:
closing_cash_2019

NameError: name 'closing_cash_2019' is not defined